# 19.5.3 Support Vector Machine Vs. Deep Learning Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
tele_df = pd.read_csv('Resources/bank_telemarketing.csv')
tele_df.head()

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


### Process the data

In [2]:
# Generate our categorical variable list
tele_cat = tele_df.dtypes[tele_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

In [3]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encoded_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

# Add the encoded variable names to the dataframe
encoded_df.columns = enc.get_feature_names(tele_cat)
encoded_df.head()

C:\Users\Jarvinia\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [4]:
# Merge one-hot encoded features and drop the originals
tele_df = tele_df.merge(encoded_df, left_index=True, right_index=True).drop(columns=tele_cat, axis=1)
tele_df.head()

,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [5]:
# Remove loan status target from features data
y = tele_df["Subscribed_yes"].values
X = tele_df.drop(columns=["Subscribed_yes", "Subscribed_no"]).values

# Split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### SVMs

In [6]:
# Create the SVM model
svm = SVC(kernel="linear")

In [7]:
# Train the model
svm.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [8]:
# Predict and evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f"SVM model accuracy: {accuracy_score(y_test, y_pred):.3f}")

SVM model accuracy: 0.873


Looking at the output of our SVM model, the model was able to correctly predict the customers who subscribed roughly 87% of the time, which is a respectable first-pass model. 

### Deep Learning Model

In [9]:
len(X_train_scaled)

22857

In [14]:
len(X_train_scaled[0])

23

In [13]:
# Define the model - deep learning neural net
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(input_dim= len(X_train_scaled[0]), units=10, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model
nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, accuracy: {model_accuracy}")

Epoch 1/100
715/715 [==============================] - 1s 1ms/step - loss: 0.4870 - accuracy: 0.8113
Epoch 2/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3716 - accuracy: 0.8733
Epoch 3/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3692 - accuracy: 0.8734
Epoch 4/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3681 - accuracy: 0.8731
Epoch 5/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3672 - accuracy: 0.8733
Epoch 6/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3667 - accuracy: 0.8732
Epoch 7/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3662 - accuracy: 0.8736
Epoch 8/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3658 - accuracy: 0.8734
Epoch 9/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3654 - accuracy: 0.8735
Epoch 10/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3650 - accura

715/715 [==============================] - 1s 1ms/step - loss: 0.3610 - accuracy: 0.8736
Epoch 82/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3612 - accuracy: 0.8734
Epoch 83/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3611 - accuracy: 0.8736
Epoch 84/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3608 - accuracy: 0.8733
Epoch 85/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3610 - accuracy: 0.8735
Epoch 86/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3609 - accuracy: 0.8737
Epoch 87/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3608 - accuracy: 0.8734
Epoch 88/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3607 - accuracy: 0.8734
Epoch 89/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3608 - accuracy: 0.8733
Epoch 90/100
715/715 [==============================] - 1s 1ms/step - loss: 0.3607 - accuracy: 

Looking at the results of our comparative analysis, the SVM and deep learning models both achieved a predictive accuracy around 87%. 

Additionally, both models take similar amounts of time to train on the input data. The only noticeable difference between the two models is implementation—the amount of code required to build and train the SVM is notably less than the comparable deep learning model. As a result, many data scientists will prefer to use SVMs by default, then turn to deep learning models, as needed.